## splitting a domain into a tile of stretched grids

Requires all grids to be at level 0. 

Let's build a test set that we will split further:

In [61]:
import numpy as np 

# initial gridding with variable widths 

x_wids = np.linspace(0, 1, 40)
y_wids = np.linspace(0, 1, 80)
z_wids = np.linspace(0, 1, 100)

x = np.cumsum(x_wids)
y = np.cumsum(y_wids)
z = np.cumsum(z_wids)

# rescale domain from 0 to 1

x = x / x.max()  # cell edges
x_wids = x[1:] - x[:-1] # cell widths

y = y / y.max()
y_wids = y[1:] - y[:-1]

z = z / z.max()
z_wids = z[1:] - z[:-1]


# assuming data is defined at cell centers 

shape = (len(z_wids), len(y_wids), len(x_wids))
data = {'density': np.random.random(shape)} 
print(shape)

(99, 79, 39)


In [62]:
x_wids, x

(array([0.00128205, 0.0025641 , 0.00384615, 0.00512821, 0.00641026,
        0.00769231, 0.00897436, 0.01025641, 0.01153846, 0.01282051,
        0.01410256, 0.01538462, 0.01666667, 0.01794872, 0.01923077,
        0.02051282, 0.02179487, 0.02307692, 0.02435897, 0.02564103,
        0.02692308, 0.02820513, 0.02948718, 0.03076923, 0.03205128,
        0.03333333, 0.03461538, 0.03589744, 0.03717949, 0.03846154,
        0.03974359, 0.04102564, 0.04230769, 0.04358974, 0.04487179,
        0.04615385, 0.0474359 , 0.04871795, 0.05      ]),
 array([0.        , 0.00128205, 0.00384615, 0.00769231, 0.01282051,
        0.01923077, 0.02692308, 0.03589744, 0.04615385, 0.05769231,
        0.07051282, 0.08461538, 0.1       , 0.11666667, 0.13461538,
        0.15384615, 0.17435897, 0.19615385, 0.21923077, 0.24358974,
        0.26923077, 0.29615385, 0.32435897, 0.35384615, 0.38461538,
        0.41666667, 0.45      , 0.48461538, 0.52051282, 0.55769231,
        0.59615385, 0.63589744, 0.67692308, 0.71923077, 0.

domain decomposition: splitting by element still makes sense, but we need to correctly calculate edges. 

The function for decomposing, `yt.utilities.decompose.split_array`, assumes uniform spacing:

```python
def split_array(gle, gre, shape, psize):
    """Split array into px*py*pz subarrays."""
    n_d = np.array(shape, dtype=np.int64)
    dds = (gre - gle) / shape
    left_edges = []
    right_edges = []
    shapes = []
    slices = []
    for i in range(psize[0]):
        for j in range(psize[1]):
            for k in range(psize[2]):
                piece = np.array((i, j, k), dtype=np.int64)
                lei = n_d * piece // psize
                rei = n_d * (piece + np.ones(3, dtype=np.int64)) // psize
                lle = gle + lei * dds
                lre = gle + rei * dds
                left_edges.append(lle)
                right_edges.append(lre)
                shapes.append(rei - lei)
                slices.append(np.s_[lei[0] : rei[0], lei[1] : rei[1], lei[2] : rei[2]])

    return left_edges, right_edges, shapes, slices
```

so let's first right this differently... first, it's OK to still use `get_psize` to get the number of grids since this works entirely in index space. So if we want to balance by number of elements in each grid, let's do that:

In [63]:
from yt.utilities.decompose import get_psize 

In [64]:
psize = get_psize(np.array(shape), 8)  # 8 subgrids (or "pieces")
print(psize)

[4 2 1]


first, a function to split the array indices:

In [65]:

def split_array_indices(shape, psize):
    """Split array into px*py*pz subarrays."""
    n_d = np.array(shape, dtype=np.int64)    
    left_edge_i = []
    right_edge_i = []
    shapes = []
    slices = []
    for i in range(psize[0]):
        for j in range(psize[1]):
            for k in range(psize[2]):
                piece = np.array((i, j, k), dtype=np.int64)
                lei = n_d * piece // psize
                rei = n_d * (piece + np.ones(3, dtype=np.int64)) // psize                
                left_edge_i.append(lei)
                right_edge_i.append(rei)
                shapes.append(rei - lei)
                slices.append(np.s_[lei[0] : rei[0], lei[1] : rei[1], lei[2] : rei[2]])

    return left_edge_i, right_edge_i, shapes, slices


In [66]:
le_i, re_i, shapes, slices = split_array_indices(np.array(shape), psize)

In [67]:
le_i

[array([0, 0, 0]),
 array([ 0, 39,  0]),
 array([24,  0,  0]),
 array([24, 39,  0]),
 array([49,  0,  0]),
 array([49, 39,  0]),
 array([74,  0,  0]),
 array([74, 39,  0])]

In [68]:
re_i

[array([24, 39, 39]),
 array([24, 79, 39]),
 array([49, 39, 39]),
 array([49, 79, 39]),
 array([74, 39, 39]),
 array([74, 79, 39]),
 array([99, 39, 39]),
 array([99, 79, 39])]

In [69]:
shapes

[array([24, 39, 39]),
 array([24, 40, 39]),
 array([25, 39, 39]),
 array([25, 40, 39]),
 array([25, 39, 39]),
 array([25, 40, 39]),
 array([25, 39, 39]),
 array([25, 40, 39])]

so we see the index ranges for each grid. now we need to get the left and right edges of each grid, along with the internal node positions. For a single grid, this would be:

so we see the index ranges for each grid. now we need to get the left and right edges of each grid, along with the internal node positions

In [58]:
grid_i = 3

left_edge = (z[le_i[grid_i][0]], y[le_i[grid_i][1]], x[le_i[grid_i][2]])
right_edge = (z[re_i[grid_i][0]-1], y[re_i[grid_i][1]-1], x[re_i[grid_i][2]-1])



In [59]:
left_edge

(0.06060606060606061, 0.24489795918367346, 0.0)

In [60]:
right_edge

(0.23757575757575758, 0.96, 0.9)

In [ ]:

def split_array_indices(cell_widths, shape, psize):
    """Split array into px*py*pz subarrays."""
    n_d = np.array(shape, dtype=np.int64)    
    left_edge_i = []
    right_edge_i = []
    shapes = []
    slices = []
    for i in range(psize[0]):
        for j in range(psize[1]):
            for k in range(psize[2]):
                piece = np.array((i, j, k), dtype=np.int64)
                lei = n_d * piece // psize
                rei = n_d * (piece + np.ones(3, dtype=np.int64)) // psize
                
                
                le_0 = cell_widths[0][:lei]
                left_edge_i.append(lei)
                right_edge_i.append(rei)
                shapes.append(rei - lei)
                slices.append(np.s_[lei[0] : rei[0], lei[1] : rei[1], lei[2] : rei[2]])

    return left_edge_i, right_edge_i, shapes, slices


In [78]:
b = z_wids[:0] 


array([], dtype=float64)

In [95]:
def get_dds_i(wid_i, lei_i, rei_i):
    dds_i = wid_i[lei_i:rei_i]
    if lei_i > 0: 
        
        offset = np.array(wid_i[0:lei_i-1].sum())
    else:
        offset = np.array(wid_i[0])
    le = offset - dds_i[0]/2.
    re = le + dds_i[lei_i:rei_i].sum() + dds_i[-1]/2.  # not sure about the +/- 1/2
    return dds_i, le, re                

In [100]:
grid_i = 0
dds_i_0, le_i_0, re_i_0 = get_dds_i(z_wids, le_i[grid_i][0], re_i[grid_i][0])
dds_i_1, le_i_1, re_i_1 = get_dds_i(y_wids, le_i[grid_i][1], re_i[grid_i][1])
dds_i_2, le_i_2, re_i_2 = get_dds_i(x_wids, le_i[grid_i][2], re_i[grid_i][2])

In [97]:
dds_0_0

array([0.00020202, 0.00040404, 0.00060606, 0.00080808, 0.0010101 ,
       0.00121212, 0.00141414, 0.00161616, 0.00181818, 0.0020202 ,
       0.00222222, 0.00242424, 0.00262626, 0.00282828, 0.0030303 ,
       0.00323232, 0.00343434, 0.00363636, 0.00383838, 0.0040404 ,
       0.00424242, 0.00444444, 0.00464646, 0.00484848])

In [98]:
le_0_0

0.00010101010101010102

In [99]:
re_0_0

0.06313131313131312

In [ ]:
le = np.array(le_i_0, le_i_1, le_i_2)
re = np.array(re_i_0, re_i_1, re_i_2)